In [ ]:
!pip install rasterio
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definir caminho da pasta no Google Drive
folder_path = "/content/drive/MyDrive/mapbiomasgdalubuntu"  # Modifique conforme sua pasta no Drive

# Listar arquivos GeoTIFF na pasta
tif_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif') and f.startswith('mapbiomas_')])
#tif_files = [f for f in os.listdir(folder_path) if f.endswith('.tif') and f.startswith('mapbiomas_')]

# Criar rasters binários para soja e natureza
for file in tif_files:
    year = file.split('_')[-1].split('.')[0]  # Extrai o ano do nome do arquivo
    file_path = os.path.join(folder_path, file)

    # Abrir o raster original
    with rasterio.open(file_path) as src:
        profile = src.profile  # Pega metadados do raster
        data = src.read(1)  # Lê a primeira (e única) banda
_2
        # Criar máscara binária para classes naturais (3, 4, 11, 12)
        natureza_bin = np.where(np.isin(data, [3, 4, 11, 12]), 1, 0).astype(np.uint8)

        # Atualizar o perfil para salvar como uint8 (valores 0 ou 1)
        profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

        # Salvar o raster binário das classes naturais
        natureza_output_path = os.path.join(folder_path, f"natureza_{year}.tif")
        with rasterio.open(natureza_output_path, 'w', **profile) as dst:
            dst.write(natureza_bin, 1)

    print(f"Processado: {file} -> natureza_{year}.tif")

print("Processamento concluído!")

Processado: mapbiomas_1985.tif -> natureza_1985.tif
Processado: mapbiomas_1986.tif -> natureza_1986.tif
Processado: mapbiomas_1987.tif -> natureza_1987.tif
Processado: mapbiomas_1988.tif -> natureza_1988.tif
Processado: mapbiomas_1989.tif -> natureza_1989.tif
Processado: mapbiomas_1990.tif -> natureza_1990.tif
Processado: mapbiomas_1991.tif -> natureza_1991.tif
Processado: mapbiomas_1992.tif -> natureza_1992.tif
Processado: mapbiomas_1993.tif -> natureza_1993.tif
Processado: mapbiomas_1994.tif -> natureza_1994.tif
Processado: mapbiomas_1995.tif -> natureza_1995.tif
Processado: mapbiomas_1996.tif -> natureza_1996.tif
Processado: mapbiomas_1997.tif -> natureza_1997.tif
Processado: mapbiomas_1998.tif -> natureza_1998.tif
Processado: mapbiomas_1999.tif -> natureza_1999.tif
Processado: mapbiomas_2000.tif -> natureza_2000.tif
Processado: mapbiomas_2001.tif -> natureza_2001.tif
Processado: mapbiomas_2002.tif -> natureza_2002.tif
Processado: mapbiomas_2003.tif -> natureza_2003.tif
Processado: 

In [ ]:
#novo raster idade desmatamento
import rasterio
import numpy as np
import os
import pandas as pd
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from rasterio.transform import xy

# Caminhos dos arquivos
raster_folder = "/content/drive/MyDrive/mapbiomasgdalubuntu/"
natural_vegetation_files = sorted([os.path.join(raster_folder, f) for f in os.listdir(raster_folder) if f.startswith("natureza_") and f.endswith(".tif")])
output_deforestation_age_path = os.path.join(raster_folder, "deforestation_age_colab.tif")

# Definir intervalo de anos
anos = list(range(1985, 2024))  # 1985 a 2023
num_anos = len(anos)

# Abrir o primeiro raster para definir metadados
with rasterio.open(natural_vegetation_files[0]) as src:
    meta = src.meta.copy()
    altura, largura = src.shape

# Criar um array para armazenar a idade do desmatamento
idade_desmate = np.zeros((altura, largura), dtype=np.uint8)

# Inicializar com 39 os pixels que já eram 0 em 1985 (desmatados antes do início da série)
with rasterio.open(natural_vegetation_files[0]) as src:
    img_anterior = src.read(1)
    idade_desmate[img_anterior == 0] = 39

# Percorrer a série temporal de 1985 a 2023
for i in range(1, num_anos):
    ano_atual = anos[i]
    idade_atual = num_anos - i  # 1985 tem idade 39, 2023 tem idade 1

    with rasterio.open(natural_vegetation_files[i]) as src:
        img_atual = src.read(1)

        # Detectar mudança de 1 -> 0 (desmatamento)
        desmatado = (img_anterior == 1) & (img_atual == 0) & (idade_desmate == 0)
        idade_desmate[desmatado] = idade_atual

        # Atualizar img_anterior para a próxima iteração
        img_anterior = img_atual.copy()

    print(f"Processado: {ano_atual}, pixels de desmatamento identificados: {np.sum(desmatado)}")

# Atualizar metadados
meta.update({
    "dtype": 'uint8',
    "count": 1,
    "nodata": 0
})

# Salvar raster de idade do desmatamento
with rasterio.open(output_deforestation_age_path, "w", **meta) as dst:
    dst.write(idade_desmate, 1)

# Exibir resultado final
plt.figure(figsize=(10, 6))
plt.imshow(idade_desmate, cmap='viridis')
plt.colorbar(label="Idade do desmatamento (anos)")
plt.title("Primeiro ano de desmatamento detectado")
plt.show()

print("Raster de idade do desmatamento criado com sucesso!")


Processado: 1986, pixels de desmatamento identificados: 78379588
Processado: 1987, pixels de desmatamento identificados: 64047431
Processado: 1988, pixels de desmatamento identificados: 59533124
Processado: 1989, pixels de desmatamento identificados: 53067186
Processado: 1990, pixels de desmatamento identificados: 44888598
Processado: 1991, pixels de desmatamento identificados: 44656479
Processado: 1992, pixels de desmatamento identificados: 49132862
Processado: 1993, pixels de desmatamento identificados: 43337310
Processado: 1994, pixels de desmatamento identificados: 46338061
Processado: 1995, pixels de desmatamento identificados: 52424744
Processado: 1996, pixels de desmatamento identificados: 45747325
Processado: 1997, pixels de desmatamento identificados: 44078362
Processado: 1998, pixels de desmatamento identificados: 43098300
Processado: 1999, pixels de desmatamento identificados: 42347400
Processado: 2000, pixels de desmatamento identificados: 39069647
Processado: 2001, pixels 